# Data Management

### Imports

In [5]:
import sys
from pathlib import Path
import re
import csv

import pandas as pd
from IPython.display import display, Markdown

### Define Paths

In [6]:
CWD = Path.cwd()
ROOT = CWD.parent if CWD.name.lower() == "notebooks" else CWD

DATA = ROOT / "data"
RAW = DATA / "raw"
PROCESSED = DATA / "processed"

RAW.mkdir(parents=True, exist_ok=True)
PROCESSED.mkdir(parents=True, exist_ok=True)

print("Python:", sys.version.split()[0])
print("CWD:", CWD)
print("ROOT:", ROOT)
print("RAW:", RAW)
print("PROCESSED:", PROCESSED)

Python: 3.11.9
CWD: C:\Users\danci\Interconnection-Queue-Intelligence\Interconnection-Queue-Intelligence\notebooks
ROOT: C:\Users\danci\Interconnection-Queue-Intelligence\Interconnection-Queue-Intelligence
RAW: C:\Users\danci\Interconnection-Queue-Intelligence\Interconnection-Queue-Intelligence\data\raw
PROCESSED: C:\Users\danci\Interconnection-Queue-Intelligence\Interconnection-Queue-Intelligence\data\processed


### Helpers

In [7]:
def looks_like_header_cell(x) -> bool:
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return False
    s = str(x).strip()
    if s == "":
        return False
    if re.fullmatch(r"[-+]?\d+(\.\d+)?", s):
        return False
    return bool(re.search(r"[A-Za-z]", s))


def detect_header_row(preview_df: pd.DataFrame, min_nonnull: int = 4) -> int:
    best_i, best_score = 0, float("-inf")

    for i in range(len(preview_df)):
        row = preview_df.iloc[i]
        nonnull = row.dropna().tolist()
        if len(nonnull) < min_nonnull:
            continue

        headerish = sum(looks_like_header_cell(v) for v in nonnull)
        norm = [str(v).strip().lower() for v in nonnull]
        dup_penalty = len(norm) - len(set(norm))

        score = len(nonnull) + 2 * headerish - 1.5 * dup_penalty
        if score > best_score:
            best_score = score
            best_i = i

    return best_i


def inspect_excel(path: Path, max_scan_rows: int = 60, head_rows: int = 5):
    display(Markdown(f"**File:** `{path.name}`"))
    if not path.exists():
        display(Markdown("**Missing file** (not found in `data/raw/`)."))
        return

    try:
        xls = pd.ExcelFile(path)
    except Exception as e:
        display(Markdown(f"**Failed to open Excel:** `{e}`"))
        return

    sheets = xls.sheet_names
    display(Markdown(f"**Sheets:** {len(sheets)}"))
    display(pd.DataFrame({"sheet": sheets}))

    for sheet in sheets:
        display(Markdown(f"#### Sheet: `{sheet}`"))

        try:
            preview = pd.read_excel(path, sheet_name=sheet, header=None, nrows=max_scan_rows)
        except Exception as e:
            display(Markdown(f"**Preview read failed:** `{e}`"))
            continue

        header_row = detect_header_row(preview)
        print(f"Detected header row (0-based in preview): {header_row}")

        try:
            df = pd.read_excel(path, sheet_name=sheet, header=header_row)
        except Exception as e:
            display(Markdown(f"**Header-based read failed:** `{e}`"))
            continue

        df = df.loc[:, ~df.columns.to_series().astype(str).str.match(r"^Unnamed:")].copy()

        cols = [str(c) for c in df.columns.tolist()]
        display(Markdown(f"Columns: **{len(cols)}**"))
        display(pd.DataFrame({"column": cols}))

        display(Markdown(f"Preview: `head({head_rows})`"))
        display(df.head(head_rows))

### List Raw Files

In [8]:
raw_files = sorted([p for p in RAW.iterdir() if p.is_file() and not p.name.startswith("~$")])
display(pd.DataFrame({"file": [p.name for p in raw_files]}))

,file
0,cluster-15-interconnection-requests.xlsx
1,PreliminaryCluster14ProjectListasofMay20-2021....
2,publicqueuereport.xlsx


## Inspect Public Queue

In [9]:
PUBLIC_QUEUE_FILE = "publicqueuereport.xlsx"

path = RAW / PUBLIC_QUEUE_FILE
inspect_excel(path)

**File:** `publicqueuereport.xlsx`

**Sheets:** 3

,sheet
0,Grid GenerationQueue
1,Completed Generation Projects
2,Withdrawn Generation Projects


#### Sheet: `Grid GenerationQueue`

Detected header row (0-based in preview): 3


...

Detected header row (0-based in preview): 3


Columns: **31**

,column
0,Project Name - Confidential
1,Queue Position
2,Interconnection Request\nReceive Date
3,Queue Date
4,Application Status
5,Withdrawn Date
6,Study\nProcess
7,Type-1
8,Type-2
9,Type-3


Preview: `head(5)`

,Project Name - Confidential,Queue Position,Interconnection Request\nReceive Date,Queue Date,Application Status,Withdrawn Date,Study\nProcess,Type-1,Type-2,Type-3,...,Utility,Station or Transmission Line,Proposed\nOn-line Date\n(as filed with IR),Current\nOn-line Date,Feasibility Study or Supplemental Review,System Impact Study or \nPhase I Cluster Study,Facilities Study (FAS) or \nPhase II Cluster Study,Optional Study\n(OS),Interconnection Agreement \nStatus,Reason for Withdrawal
0,ENCINA GENERATING PROJECT (PH. 1 AND 2),5,2000-08-09,2000-08-09 07:00:00,WITHDRAWN,NaT,Pre- Amend. 39,Combined Cycle,NaN,NaN,...,SDGE,Encina Power Plant Switchyard,2003-06-30 07:00:00,2008-06-01 07:00:00,NaN,Complete,NaN,NaN,NaN,NaN
1,TESLA POWER PLANT,6,2007-08-24,2000-08-23 07:00:00,WITHDRAWN,2011-06-16 17:56:36,Serial LGIP,Combined Cycle,NaN,NaN,...,PGAE,Tesla Sub 230kV Bus E,2008-06-01 07:00:00,2014-11-30 08:00:00,Waived,Complete,Complete,Complete,NaN,NaN
2,SAN DIEGO COMMUNITY POWER GENERATING STATION,8,2000-11-28,2000-11-28 08:00:00,WITHDRAWN,NaT,Pre- Amend. 39,Combined Cycle,NaN,NaN,...,SDGE,Sycamore Canyon Substation,2004-06-01 07:00:00,2010-12-31 08:00:00,NaN,Complete,Re-Study,NaN,In Progress,NaN
3,MORRO BAY MODERNIZATION PROJECT,9,2000-12-01,2000-12-01 08:00:00,WITHDRAWN,NaT,Pre- Amend. 39,Combined Cycle,NaN,NaN,...,PGAE,Morro Bay Substation,2008-01-01 08:00:00,2008-01-01 08:00:00,NaN,Complete,Complete,NaN,Executed,NaN
4,AVENAL ENERGY PROJECT,10,2001-05-02,2001-05-02 07:00:00,WITHDRAWN,NaT,Pre- Amend. 39,Combined Cycle,NaN,NaN,...,PGAE,Gates Substation (Arco - Gates 230 kV line),2009-01-01 08:00:00,2009-07-01 07:00:00,NaN,Complete,Complete,NaN,Executed,NaN


## Inspect Cluster 15

In [11]:
CLUSTER15_FILE = "cluster-15-interconnection-requests.xlsx"

path = RAW / CLUSTER15_FILE
inspect_excel(path)

**File:** `cluster-15-interconnection-requests.xlsx`

**Sheets:** 2

,sheet
0,Cluster 15
1,Withdrawn


#### Sheet: `Cluster 15 `

Detected header row (0-based in preview): 0


...

Detected header row (0-based in preview): 0


Columns: **21**

,column
0,Queue Number
1,Project Number
2,Project Name
3,Generation/Fuel 1
4,NET MW 1
5,Generation/Fuel 2
6,NET MW 2
7,Generation/Fuel 3
8,NET MW 3
9,NET MW POI


Preview: `head(5)`

,Queue Number,Project Number,Project Name,Generation/Fuel 1,NET MW 1,Generation/Fuel 2,NET MW 2,Generation/Fuel 3,NET MW 3,NET MW POI,...,Project State,Study Area,PTO,POI,Voltage kV,Requested COD,Queue Date,Application Date,Withdrawal Date,Service Type
0,2229,54899,Clay Flats,Storage/Battery,437.080,NaN,NaN,NaN,NaN,425.0,...,CA,PG&E FRESNO,LSPC,MANNING 500 kV,500,2030-10-01,2025-02-12,2024-11-22,2025-04-23,Energy Only Requested
1,2202,55018,Gibson,Storage/Battery,154.598,NaN,NaN,NaN,NaN,150.0,...,CA,PG&E FRESNO,PGAE,MERCY SPRINGS SW STA 70 kV,70,2028-04-14,2025-02-12,2024-11-15,2025-04-24,Energy Only Requested
2,2283,54729,Amargosa SEZ,Photovoltaic/Solar,510.350,Storage/Battery,508.19,NaN,NaN,500.0,...,NV,SCE EOP,GLW,BEATTY 230 kV,230,2030-12-01,2025-02-12,2024-11-18,2025-04-25,Merchant- Full Capacity Deliverability Status ...
3,2228,54946,Grapevine 2,Storage/Battery,505.100,NaN,NaN,NaN,NaN,500.0,...,NV,SCE EOP,SCE,ELDORADO 230 kV,230,2028-06-01,2025-02-12,2024-11-18,2025-04-25,Merchant- Full Capacity Deliverability Status ...
4,2290,54668,Ashton Energy Center,Photovoltaic/Solar,1026.670,Storage/Battery,1022.47,NaN,NaN,1000.0,...,NV,SCE EOP,GLW,VALLEY SWITCH 230 kV,230,2032-12-01,2025-02-12,2024-11-18,2025-04-25,Merchant- Full Capacity Deliverability Status ...


## Inspect Cluster 14

In [12]:
CLUSTER14_FILE = "PreliminaryCluster14ProjectListasofMay20-2021.xlsx"

path = RAW / CLUSTER14_FILE
inspect_excel(path)

**File:** `PreliminaryCluster14ProjectListasofMay20-2021.xlsx`

**Sheets:** 2

,sheet
0,Notes
1,C14 Prelim Cluster Overview


#### Sheet: `Notes`

Detected header row (0-based in preview): 0


...

Detected header row (0-based in preview): 0


Columns: **12**

,column
0,PTO
1,Affected PTO
2,Area
3,Total MW @ POI
4,Technology\n#1
5,MW-T1
6,Technology\n#2
7,MW-T2
8,Technology\n#3
9,MW-T3


Preview: `head(5)`

,PTO,Affected PTO,Area,Total MW @ POI,Technology\n#1,MW-T1,Technology\n#2,MW-T2,Technology\n#3,MW-T3,POI,Voltage\n(kV)
0,DCRT,SCE,Eastern,500.0,Battery,516.7638,Solar PV,516.7638,NaN,NaN,Cielo Azul Substation,500.0
1,DCRT,SCE,Eastern,350.0,Battery,350.0000,Solar PV,350.0000,NaN,NaN,Delaney Substation,500.0
2,DCRT,SCE,Eastern,2000.0,Battery,2000.0000,NaN,NaN,NaN,NaN,Cielo Azul Substation,500.0
3,DCRT,SCE,Eastern,350.0,Solar PV,357.5300,NaN,NaN,NaN,NaN,Delaney-Colorado River,500.0
4,DCRT,SCE,Eastern,700.0,Battery,718.8100,Solar PV,718.8100,NaN,NaN,Delaney-Colorado River,500.0
